# Imports

In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import zipfile
from skimage.measure import compare_ssim as ssim

import tensorflow as tf
import keras.backend as K
from keras.layers import Lambda, Input, Average
from keras.models import Model

Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import MCS2018

net = MCS2018.Predictor(0)

In [3]:
from utils.preprocessing import *
from utils.common import load_data, gpu_config, FaceNetFinetuned, ModelEnsemble, Attacker

### Load and preprocess data

In [4]:
imgs_path = 'data/imgs/'
train_imgs_source, train_imgs_target, desc_source, desc_target = load_data(
    path_to_desc='data/train_descriptors_augmented20.npz')

# augment targets with mirrored images
train_imgs_target = np.hstack((train_imgs_target, train_imgs_target[:,:,:,::-1,:].copy()))

### Make an ensemble of neural networks

In [5]:
gpu_id = 0
config = gpu_config(gpu_id)
sess = tf.Session(config=config)
K.set_session(sess)

In [6]:
neural_net1 = FaceNetFinetuned(path_to_models='models/', model_name='bs256_adversarial_aug_bn.h5', bn=True)
neural_net2 = FaceNetFinetuned(path_to_models='models/', model_name='bs256_adversarial_aug_bn2.h5', bn=True)
neural_net3 = FaceNetFinetuned(path_to_models='models/', model_name='bs256_adversarial_aug_bn3.h5', bn=True)

/workspace/virtualenv36/lib/python3.6/site-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [7]:
neural_net = ModelEnsemble([neural_net1, neural_net2, neural_net3])

### Train attacker for submit

In [8]:
submit_path = 'submit_main/'

if not os.path.isdir(submit_path):
    os.mkdir(submit_path)

source_imgs_list = pd.read_csv('data/submit_list.csv')
learning_rates = np.load('learning_rates.npz')['lr']

In [9]:
attacker = Attacker(net, neural_net, sess)

In [10]:
print ("Ready attacker one!")
print ("-------------------------------")

scores = []
for idx, img_path in tqdm(enumerate(source_imgs_list.path)):
    
    i, j = idx // 5, idx % 5
    
    lr = learning_rates[idx]
    img, targets = train_imgs_source[i, j], desc_target[i, :]
    pimg, score = attacker.nesterov_attack(img, targets, maxiter=40, alpha0=lr, decay=0.98, tol=2e-4)
    pimg = np.clip(denormalize(pimg), 0, 1) 
    png_img = Image.fromarray(np.array(pimg*255, dtype=np.uint8))
    png_img.save(os.path.join(submit_path, img_path))
    
    if (idx % 5 == 0 and idx > 0):
        print ("--------------- Images", str(idx//5), "----------------")
        print ("Nesterov momentum score:", round(np.mean(scores), 5))
    scores.append(score)

Ready attacker one!
-------------------------------


--------------- Images 1 ----------------
Nesterov momentum score: 1.11976
--------------- Images 2 ----------------
Nesterov momentum score: 1.12336
--------------- Images 3 ----------------
Nesterov momentum score: 1.1036
--------------- Images 4 ----------------
Nesterov momentum score: 1.06686



### Calculate descriptors and  check generated attacks for SSIM violation

In [11]:
submit_path = 'submit_main/'

In [12]:
def check_image(img_path):
    
    img1 = Image.open(submit_path+img_path)
    img2 = img_to_crop(Image.open(imgs_path+img_path.replace('png', 'jpg')))
    desc = net.submit(crop_to_tensor(img1))

    ssim_ = ssim(np.array(img1), np.array(img2), multichannel=True)
    return desc, (ssim_ < 0.95)

In [13]:
res_desc = np.zeros((5000, 512), dtype=np.float32)
for idx, img_path in tqdm(enumerate(source_imgs_list.path)):
    
    desc, ssim_is_bad = check_image(img_path)
    
    tol = 3e-4  
    while ssim_is_bad: 
        print ("Bitch!", "image =", idx)
        i, j = idx // 5, idx % 5
        img, targets = train_imgs_source[i, j], desc_target[i, :]
        pimg, score = attacker.nesterov_attack(img, targets, maxiter=40, alpha0=0.95, tol=tol)
        png_img = Image.fromarray(np.array(pimg*255, dtype=np.uint8))
        png_img.save(os.path.join(submit_path, img_path))
        desc, ssim_is_bad = check_image(img_path)
        tol += 1e-4
            
    res_desc[idx] = desc.copy()

In [14]:
descriptors_path = submit_path + 'descriptors.npy'
np.save(descriptors_path, res_desc)

In [15]:
diff = (desc_target[:,:20])[:,:,None,:] - res_desc.reshape((1000, 5, 512))[:,None,:,:]
score = np.mean(np.linalg.norm(diff, axis=-1))
print ("Loss on public target images:", score)

Loss on public target images: 0.9530307


### Archive everything for submit

In [16]:
source_imgs_list_png = source_imgs_list.copy()
source_imgs_list_png.path = source_imgs_list_png.path.apply(lambda x: x[:-3]+'png')

if not os.path.isdir('./submits/'):
    os.makedirs('./submits')
submit_file = './submits/' + submit_path[:-1] + '.zip'
    
with zipfile.ZipFile(submit_file,'w') as myzip:
    for img_name in tqdm(source_imgs_list_png.path.values, desc='archive'):
        myzip.write(os.path.join(submit_path[:-1], img_name), arcname=img_name)
    myzip.write(descriptors_path, arcname='descriptors.npy')